In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

In [2]:
df_train = json.load(open(r"C:\Users\22794\Desktop\SML A2\SMLproject2\data\backup\train.json"))
df_test = pd.read_json(r"C:\Users\22794\Desktop\SML A2\SMLproject2\data\downsamp\test.json")

In [3]:
coauthor = []
text = []
venue = []
df = []

for i, each in enumerate(df_train):
    venue_onehot = np.zeros(465)
    if each['venue'] != '':
        venue_onehot[[each['venue']]] = 1
    
    coauthor_onehot = np.zeros(21146)
    for j in each['authors']:
        if j < 100:
            coauthor_onehot[j] = 1
    
    text_onehot = np.zeros(4999)
    for k in each['abstract']:
        text_onehot[k-1] = 1
    
    textual = np.concatenate([venue_onehot, coauthor_onehot, text_onehot], axis=0)
    df.append(textual)

In [4]:
labels = []
for i, each in enumerate(df_train):
    label = np.zeros(100)
    have_coauthor = [i for i in each['authors'] if i < 100]
    if len(have_coauthor) == 0:
        label[-1] == 1
    else:
        label[have_coauthor] = 1
    labels.append(label)

In [5]:
train, valid, labels_train, labels_valid = train_test_split(df, labels, test_size=0.2, random_state=51)

### Multi-label Support Vector Machine

In [ ]:
from sklearn.svm import LinearSVC, SVC, NuSVC
from sklearn.multioutput import MultiOutputClassifier

svm = SVC()
multilabel_classifier = MultiOutputClassifier(svm, n_jobs=-1)
multilabel_classifier = multilabel_classifier.fit(train, labels_train)

In [ ]:
import pickle
filename = 'onehot_svm.sav'
pickle.dump(svm, open(filename, 'wb'))

In [ ]:
train_predictions = multilabel_classifier.predict(data_train)

f1_score(labels_train, train_predictions, average='samples')

In [ ]:
predictions = multilabel_classifier.predict(data_valid)

f1_score(labels_valid, predictions, average='samples')